# Ingesting an Image

This example shows how to ingest a grayscale image and save the results locally.

## Getting the Data

Before we can begin with the ingest, we must first download the data from S3. This curl command will download a file from S3 and save it to your `/tmp` direcotry.

**A side note**: Files can be retrieved directly from S3 using the methods shown in this tutorial. However, this could not be done in this instance due to permission requirements needed to access the file.

In [ ]:
!curl -o /tmp/cropped.tif https://s3.amazonaws.com/geopyspark-test/example-files/cropped.tif

## The Code

With our file downloaded we can begin the ingest.

In [ ]:
from geopyspark import geopyspark_conf
from geopyspark.geotrellis import catalog, geotiff
from geopyspark.geotrellis.constants import LayerType, LayoutScheme

from pyspark import SparkContext

### Setting Up the SparkContext

The first thing one needs to do when using GeoPySpark is to setup `SparkContext`. Because GeoPySpark is backed by Spark, the `pysc` is needed to initialize our starting classes.

For those that are already familiar with Spark, you may already know there are multiple ways to create a `SparkContext`. When working with GeoPySpark, it is advised to create this instance via `SparkConf`. This is where the `geopyspark_conf` comes into play. There are numerous settings for `SparkConf`, and some **have** to be set a certain way in order for GeoPySpark to work. Thus, `geopyspark_conf` was created as way for a user to set the basic parameters without having to worry about setting the other requirements.

In [ ]:
conf = geopyspark_conf(master="local[*]", appName="ingest-example")
pysc = SparkContext(conf=conf)

### Reading in the Data

After the creation of `pysc`, we can now read in the data. For this example, we will be reading in a single GeoTiff that contains spatial data. Hence, why we set the `layer_type` to `LayerType.SPATIAL`.

In [ ]:
raster_layer = geotiff.get(pysc=pysc,
                           layer_type=LayerType.SPATIAL,
                           uri="file:///tmp/cropped.tif")

### Collecting the Metadata

Now that the raster layer is loaded in, it is time to tile data. To tile the rasters, we must first decide on how they should be laid out and assorted. We can create metadata for `raster_layer` that will contain information on this layout.

`collect_metadata` method has various parameters that can be set which allow a user to customize how the tiles are sorted. For this tutorial, we will just use the default parameters.

In [ ]:
metadata = raster_layer.collect_metadata()
metadata

### Tiling the Data

With the metadata collected, it is now time to format the data within the layer to our desired layout. The aptly named, `tile_to_layout`, method will cut and arrange the rasters in the layer to the layout within the metadata; giving us a new class instance of `TiledRasterLayer`.

In [ ]:
tiled_raster_layer = raster_layer.tile_to_layout(metadata)
tiled_raster_layer

You may have noticed in the output above, but `TiledRasterLayer` has an attribute `zoom_level`. This corresponds to a level of a pyramid used in TMS servers. When first created, a `TiledRasterLayer` will have a `zoom_level` of `None`, but this can change depending on how the data within the layer is altered.

With our tiled data, we'd like to make a tile server from it and show it in on a map. Therefore, we have to make sure that the tiles within the layer are in the right projected. In this case, Web Mercator. Using the `reproject` method we can choose our target `crs` and `scheme`.

In [ ]:
# crs before reprojection
tiled_raster_layer.layer_metadata.crs

In [ ]:
reprojected_layer = tiled_raster_layer.reproject(target_crs=3857,
                                                 scheme=LayoutScheme.ZOOM)

In [ ]:
# crs after reprojection
reprojected_layer.layer_metadata.crs

#### A change of Zoom

Reprojection is one of the methods that produce a `TiledRasterLayer` with a `zoom_level`.

In [ ]:
reprojected_layer.zoom_level

During the reprojection step, there are other parameters that can be supplied so that the resulting `TiledRasterLayer` will have a certain zoom level. For this tutorial, though, we will simply just use whatever is created.

### Pyramiding the Data

Now it's time to pyramid! With our reprojected data, we will create an instance of `Pyramid` that contains 12 `TiledRasterLayer`s. Each one having it's own `zoom_level` from 12 to 1.

In [ ]:
pyramided_layer = reprojected_layer.pyramid(start_zoom=12, end_zoom=1)

In [ ]:
pyramided_layer.levels

### Saving the Pyramid Locally

To save all of the `TiledRasterLayer`s within `pyramid_layer`, we just have to loop through values of `pyramid_layer.level` and write each layer locally.

In [ ]:
for tiled_layer in pyramided_layer.levels.values():
    catalog.write(uri="file:///tmp/ingested-image", layer_name="ingested-image", tiled_raster_rdd=tiled_layer)